In [38]:
import torch
import sys
sys.path.append('..')
from src.utils.model_utils import load_model
from tokenizers import Tokenizer
from src.data.text_dataset import TextDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [2]:
tokenizer = Tokenizer.from_file("../models/model-bpe/tokenizer.json")
model = load_model("../models/model-bpe/checkpoint_6")

In [26]:
dataset = TextDataset(
    data_dir="../data/base/test",
    tokenizer=tokenizer,
    seq_len=32
)

test_loader = DataLoader(
    dataset=dataset, 
    batch_size=1,
    shuffle=True
)

In [33]:
x, y = next(iter(test_loader))

In [28]:
print(x, y)

tensor([[27935,    90,  3782,   385,    17,  7617,  7873, 47910,   213, 11151,
           196, 13739,  1372,    29, 21966,    90,  2961, 12167,  1530,    15,
           321,   232,   190,  3872,   155,   379,  3793,  5517,  9909,  1107,
           127,   127]]) tensor([[   90,  3782,   385,    17,  7617,  7873, 47910,   213, 11151,   196,
         13739,  1372,    29, 21966,    90,  2961, 12167,  1530,    15,   321,
           232,   190,  3872,   155,   379,  3793,  5517,  9909,  1107,   127,
           127, 37882]])


In [ ]:
logits = model(x)

log_probs = F.log_softmax(logits, dim=-1)

In [35]:
print(tokenizer.decode(x[0].tolist()))

/info/zasady-wykorzystania/
Zapoznaj się z nimi, zanim udostępnisz dalej nasze książki.

Tekst opracowany na podstawie: Zofia N


In [36]:
print(tokenizer.decode(y[0].tolist()))

info/zasady-wykorzystania/
Zapoznaj się z nimi, zanim udostępnisz dalej nasze książki.

Tekst opracowany na podstawie: Zofia Nał


In [37]:
print(log_probs.shape)
predicted_tokens = log_probs.argmax(dim=-1)

decoded_text = tokenizer.decode(predicted_tokens[0].tolist())
print(decoded_text)

torch.Size([1, 32, 50000])
info/zasady-wykorzystania/
Zapoznaj się z nimi, zanim udostępnisz dalej nasze książki.

Tekst opracowany na podstawie: Zo nogach Nającej


In [41]:
mask = (y != tokenizer.token_to_id("<pad>"))
target_log_probs = log_probs.gather(
    dim=-1,
    index=y.unsqueeze(-1)

).squeeze(-1)
masked_log_probs = target_log_probs * mask

negative_log_likelihood = -masked_log_probs.sum()
meana = negative_log_likelihood.item() / mask.sum()
torch.exp(meana)

tensor(1.8914)

In [43]:
torch.exp(target_log_probs)

tensor([[9.3797e-01, 9.9519e-01, 8.7491e-01, 9.9786e-01, 8.6851e-01, 9.9268e-01,
         9.9946e-01, 9.6742e-01, 9.5910e-01, 9.3999e-01, 9.9833e-01, 9.9898e-01,
         9.9113e-01, 9.9549e-01, 9.4893e-01, 8.5604e-01, 9.6733e-01, 9.5800e-01,
         9.6563e-01, 9.9863e-01, 9.9948e-01, 9.9954e-01, 8.6494e-01, 9.7711e-01,
         9.3038e-01, 9.9835e-01, 9.2467e-01, 9.9706e-01, 3.5313e-01, 1.7468e-03,
         7.5795e-01, 9.4050e-06]], grad_fn=<ExpBackward0>)

In [64]:
probs = F.softmax(logits, dim=-1)
print(torch.max(probs[0][0]))
print(torch.argmax(probs[0][0]))
print(probs[0][0][2810])
print(y[0][0])
print(type(y[0][0].item()))
print(tokenizer.decode([y[0][0].item()]))

tensor(0.9380, grad_fn=<MaxBackward1>)
tensor(2810)
tensor(0.9380, grad_fn=<SelectBackward0>)
tensor(2810)
<class 'int'>
info
